In [1]:
"""
参考リスト
http://mocobeta.github.io/janome/
-> Janomeの公式ドキュメント、用法が載っている
http://www.jp.undp.org/content/tokyo/ja/home/sustainable-development-goals.html
-> Sustainable Development Goals（データ出展）
"""
import codecs
from gensim import corpora, matutils
from janome.tokenizer import Tokenizer
import numpy as np
import scipy.sparse

## BoWの作成

In [2]:
# ストップワードの設定、Janome インスタンスの生成
stop_words = ["平和", "人間"] #全文書に出てくる特徴的でない単語はストップワードとして除く
tokenizer = Tokenizer()

In [3]:
# Janomeでの処理スクリプト（名詞のみ、ストップワードは除く、2文字以上の単語、数字は使用しない）
def token_generator(text):
    for text_line in text.split('\n'):
        for token in tokenizer.tokenize(text_line):
            if token.part_of_speech.split(',')[0] == '名詞' \
            and token.surface not in stop_words:
                if len(token.surface) > 1 \
                and not(token.surface.isdigit()):
                    yield token.surface

In [4]:
# データの読み込み＆形態素解析
text_processed = []
t = Tokenizer()
for i in range(17):
    file_path = "./files_step4/data/SDGs"+str(i+1)+".txt"
    with codecs.open(file_path, "r", "utf-8") as f:
        txt = f.read()
        text_processed.append(list(token_generator(txt)))

print(text_processed[0])

['形態', '貧困', '根絶', '人類', '直面', '一つ', '重要', '課題', '世界', '極度', '貧困', '半分', '以下', '減少', '多く', '基本', 'ニーズ', '世界', '以上', 'ドル', 'セント', '未満', '十分', '食料', 'きれい', '衛生', '施設', '利用', '人々', '中国', 'インド', '国々', '経済', '成長', '貧困', '進捗', '男女', '一様', '女性', '雇用', '教育', '資産', 'アクセス', '平等', '貧困', '状態', '確率', '男性', '世界', '極度', '貧困', '人々', 'アジア', 'サハラ', '以南', 'アフリカ', '地域', '進捗', '気候', '変動', '紛争', '食料', '不安', '新た', '脅威', '割合', '今後', '上昇', '持続', '可能', '開発', '目標', 'SDGs', 'たち', '開始', '取り組み', '完了', '形態', '貧困', '終止符', '大胆', 'コミットメント', 'ため', '脆弱', '状況', '人々', '対象', '基本', '資源', 'サービス', 'アクセス', '改善', '紛争', '気候', '変動', '関連', '災害', '被災', 'コミュニティ', '支援', 'こと', '必要', '貧困', '解消', '持続', '可能', '開発', 'ため', 'アジェンダ', '構成', 'グローバル', '目標', '一つ', '複数', '目標', '達成', 'ため', '包括', 'アプローチ', '必要', '不可欠']


In [5]:
# 辞書の作成と保存
dictionary = corpora.Dictionary(text_processed)
dictionary.save('./files_step4/dictionary.dict')

In [6]:
#BoW matrixの作成＆保存
corpus = [dictionary.doc2bow(doc) for doc in text_processed]
doc_matrix = matutils.corpus2csc(corpus).transpose()
scipy.sparse.save_npz('./files_step4/category_matrix.npz', doc_matrix)

In [7]:
# 中身の確認
print(doc_matrix.shape)
print(doc_matrix.toarray()[:2,:10])
print(type(doc_matrix))
print(dictionary.token2id)

(17, 593)
[[1. 1. 1. 1. 3. 2. 1. 1. 1. 1.]
 [2. 0. 5. 1. 3. 1. 0. 1. 1. 1.]]
<class 'scipy.sparse._csr.csr_matrix'>
{'SDGs': 0, 'きれい': 1, 'こと': 2, 'たち': 3, 'ため': 4, 'アクセス': 5, 'アジア': 6, 'アジェンダ': 7, 'アフリカ': 8, 'アプローチ': 9, 'インド': 10, 'グローバル': 11, 'コミットメント': 12, 'コミュニティ': 13, 'サハラ': 14, 'サービス': 15, 'セント': 16, 'ドル': 17, 'ニーズ': 18, '一つ': 19, '一様': 20, '上昇': 21, '不可欠': 22, '不安': 23, '世界': 24, '中国': 25, '人々': 26, '人類': 27, '今後': 28, '以上': 29, '以下': 30, '以南': 31, '利用': 32, '割合': 33, '包括': 34, '十分': 35, '半分': 36, '取り組み': 37, '可能': 38, '国々': 39, '地域': 40, '基本': 41, '変動': 42, '多く': 43, '大胆': 44, '女性': 45, '完了': 46, '対象': 47, '平等': 48, '形態': 49, '必要': 50, '成長': 51, '持続': 52, '支援': 53, '改善': 54, '教育': 55, '新た': 56, '施設': 57, '未満': 58, '根絶': 59, '極度': 60, '構成': 61, '気候': 62, '減少': 63, '災害': 64, '状態': 65, '状況': 66, '男女': 67, '男性': 68, '目標': 69, '直面': 70, '確率': 71, '紛争': 72, '終止符': 73, '経済': 74, '脅威': 75, '脆弱': 76, '衛生': 77, '被災': 78, '複数': 79, '解消': 80, '課題': 81, '貧困': 82, '資源': 83, '資産': 84, '進捗': 8

## Cos類似度の計算

In [8]:
# 推論ファイルの読み込み＆形態素解析
txt_inference = []
file_path = "./files_step4/data/SDGs5.txt"
with codecs.open(file_path, "r", "utf-8") as f:
    txt = f.read()
    txt_inference.append(list(token_generator(txt)))

In [9]:
# 辞書のロード＆BoW生成
dictionary = corpora.Dictionary.load('./files_step4/dictionary.dict')
corpus = [dictionary.doc2bow(doc) for doc in txt_inference]
inf_matrix = matutils.corpus2csc(corpus,\
    num_terms=len(dictionary)).transpose()

In [10]:
# カテゴリ情報BoWの読み込み＆連結
category_matrix = scipy.sparse.load_npz('./files_step4/category_matrix.npz')
target_mat = scipy.sparse.vstack([category_matrix, inf_matrix])

In [11]:
# cos類似度の計算、出力
cos_sim = np.zeros([target_mat.shape[0]])
var_SDGs = target_mat.dot(target_mat.transpose()).toarray()
for i in range(target_mat.shape[0]):
    cos_sim[i] = var_SDGs[i,-1]/(np.sqrt(var_SDGs[i, i])*\
        np.sqrt(var_SDGs[-1, -1]))   #-1を利用することで後ろから要素を取り出せる
    
np.set_printoptions(precision=3)
print(cos_sim)

[0.361 0.441 0.393 0.448 1.    0.291 0.259 0.47  0.372 0.292 0.279 0.319
 0.281 0.253 0.307 0.423 0.45  1.   ]
